In [2]:
import fitz

In [37]:
file_name = "nutrition/Nutrition-Science-and-Everyday-Application-1694655583.pdf"
# file_name = "nutrition/The Need for Professional Training in Nutrition Education and Communication_CASE STUDY REPORT.pdf"

In [32]:
with fitz.open(file_name) as doc:
    # for page in doc:
    #     print(page.get_text())
    page = doc.load_page(201)
    for table in page.find_tables():
        # print(table.to_pandas().to_json(orient="records", indent=4))
        print(table.to_pandas().to_markdown())
        # for row in table.extract():
        #     print(row)


|    | Food                                  | Standard       | Calories in Standard   | Dietary Fiber in Standard   |
|    |                                       | Portion Size   | Portion                | Portion (g)                 |
|---:|:--------------------------------------|:---------------|:-----------------------|:----------------------------|
|  0 | Shredded wheat ready-to-eat cereal    | 1-1 ¼cup       | 155-220                | 5.0-9.0                     |
|    | (various)                             |                |                        |                             |
|  1 | Wheat bran flakes ready-to-eat cereal | ¾ cup          | 90-98                  | 4.9-5.5                     |
|    | (various)                             |                |                        |                             |
|  2 | Lentils, cooked                       | ½ cup          | 115                    | 7.8                         |
|  3 | Black beans, cooked                   | ½

In [38]:
with fitz.open(file_name) as doc:
    toc = doc.get_toc()
    print(toc)
    for item in toc:
        level, title, page_num = item
        # indent = "  " * (level - 1)
        # print(f"{indent}{title} (Page: {page_num})")
        print(f"level: {level}, title: {title}, page_num: {page_num}")

[[1, 'Contents', 5], [1, 'Introduction', 9], [1, 'About the Authors', 11], [1, 'Acknowledgements', 13], [1, 'Updates Made to OER', 14], [1, 'Unit 1 – Designing A Healthy Diet', 15], [2, 'Introduction to Designing a Healthy Diet', 17], [2, 'Nutrition and Health', 19], [2, 'An Introduction to Molecules', 25], [2, 'Classification of Nutrients', 30], [2, 'Defining Nutrient Requirements: Dietary Reference Intakes', 40], [2, 'Understanding Food Labels', 48], [2, 'Tools for Achieving a Healthy Diet', 58], [1, 'Unit 2 — Nutrition Science and Information Literacy', 75], [2, 'Introduction to Nutrition Science and Information Literacy', 77], [2, 'The Scientific Method', 79], [2, 'Types of Research Studies and How To Interpret Them', 84], [2, 'Healthy Skepticism in Nutrition Science', 92], [2, 'Fact-Checking Online Health and Nutrition Information', 100], [2, 'Who Are the Experts in Nutrition Science?', 108], [2, 'Finding Accurate Sources of Health and Nutrition Information', 113], [1, 'Unit 3 – M

In [101]:
def get_section_to_idx(doc, page_num):
    toc = doc.get_toc()
    if page_num > doc.page_count:
        return "N/A"
    
    for i, (level, title, start_page_num) in enumerate(toc):
        if start_page_num <= page_num:
            if toc[-1][2] <= page_num:
                return toc[-1][1]
            elif page_num < toc[i+1][2]:
                return title
            continue
        else:
            return "cover page"
    return "N/A"

In [190]:
from langchain_core.documents import Document

def get_contents_from_pdf(file_name, doc, i: int, section: str, before_text = "") -> tuple[str, int, list[Document]]:
    contents_docs = []
    next_i = i + 1
    page = doc.load_page(i)
    text = before_text + page.get_text()
    tables = page.find_tables()
    links = page.get_links()
    # images = page.get_images()

    for table_idx, table in enumerate(tables):
        contents_docs.append(Document(
            page_content=table.to_pandas().to_markdown(),
            metadata={
                "source": file_name,
                "page": i + 1,
                "type": "table",
                "table_index_on_page": table_idx,
                "section": section
            }
        ))


    for link_idx, link in enumerate(links):
        if link['kind'] == fitz.LINK_URI:
            link_url = link['uri']
            contents_docs.append(
                Document(
                    page_content=f"페이지 {i + 1}에 외부 웹사이트 '{link_url}'로 연결되는 링크가 있습니다.",
                    metadata={
                        "source": file_name,
                        "page": i + 1,
                        "type": "hyperlink",
                        "link_url": link_url,
                        "link_index_on_page": link_idx,
                        "section": section
                    }
                )
            )


    # if text[-1] != ".":
    #     print("\n------\n", text, "\n------\n")
    #     text, next_i, add_table_docs = get_contents_from_pdf(doc, next_i, section, text)
    #     table_docs.extend(add_table_docs)
    
    text = ".\n".join(text.replace("\n", "").split("."))
    return text, next_i, contents_docs

In [191]:
with fitz.open(file_name) as doc:
    i = 32
    section = get_section_to_idx(doc, i)
    print("section:", section)
    text, next_i, table_docs = get_contents_from_pdf(file_name, doc, i, section)
    print(text)
    if table_docs:
        print("table_docs:")
        print(table_docs[0].page_content)

section: Classification of Nutrients
Major Minerals Major Function Sodium Fluid balance, nerve transmission, muscle contraction Chloride Fluid balance, stomach acid production Potassium Fluid balance, nerve transmission, muscle contraction Calcium Bone and teeth health maintenance, nerve transmission, muscle contraction, blood clotting Phosphorus Bone and teeth health maintenance, acid-base balance Magnesium Protein production, nerve transmission, muscle contraction Sulfur Protein production Trace Minerals Function Iron Carries oxygen, assists in energy production Zinc Protein and DNA production, wound healing, growth, immune system function Iodine Thyroid hormone production, growth, metabolism Selenium Antioxidant Copper Coenzyme, iron metabolism Manganese Coenzyme Fluoride Bone and teeth health maintenance, tooth decay prevention Chromium Assists insulin in glucose metabolism Molybdenum Coenzyme Table 1.
1.
 Minerals and their major functions Vitamins Vitamins are organic nutrients t

In [79]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

docs = []
with fitz.open(file_name) as doc:
    i = 0
    while i < 30:
        get 
        get_text_table_from_pdf("", doc, i, "N/A")


In [71]:
len(texts), len(texts.split(".")), len(texts.split("\n"))

(40826, 389, 389)

In [72]:
result = text_splitter.split_text(texts)
len(result)

52

In [74]:
print(result[20])

8.
 Use tools for planning a healthful diet, including MyPlate, Harvard Healthy Eating Plate, and the Dietary Guidelines for Americans.
 Image Credits: Person cooking at a table photo by Markus Winkler on Unsplash (license information) 10TAMBERLY POWELL, MS, RDNNutrition and Health WHAT IS NUTRITION? Simply put, food is the plants and animals that we eat, and nutrition is how food affects the health of the body.
 According to the Academy of Nutrition and Dietetics, “Food is essential—it provides vital nutrients for survival, and helps the body function and stay healthy.
 Food is comprised of macronutrients including protein, carbohydrate and fat that not only offer calories to fuel the body and give it energy but play specific roles in maintaining health.
 Food also supplies micronutrients (vitamins and minerals) and phytochemicals that don’t provide calories but serve a variety of critical functions to ensure the body operates optimally.


In [137]:
with fitz.open(file_name) as doc:
    page = doc.load_page(200)
    text = page.get_text()
    print(".\n".join(text.replace("\n", "").split(".")))
    print(get_section_to_idx(doc, 200))


some fruits and vegetables.
 (Psyllium fiber supplements like Metamucil are composed mainly of soluble fiber, so if you’ve ever stirred a spoonful of this into a glass of water, you’ve seen the viscous consistency characteristic of soluble fiber.
) 2.
 Insoluble Fiber – These fibers typically do not dissolve in water and are nonviscous.
 Some are fermentable by bacteria in the large intestine but to a lesser degree than soluble fibers.
 Insoluble fibers help prevent constipation, as they create a softer, bulkier stool that is easier to eliminate.
 Lignin, cellulose, and hemicellulose are common types of insoluble fibers, and food sources include wheat bran, vegetables, fruits, and whole grains.
 FOOD SOURCES OF DIETARY FIBER Since fiber provides structure to plants, fiber can be found in all whole plant foods, including whole grains (like oatmeal, barley, rice and wheat), beans, nuts, seeds, and whole fruits and vegetables.
 Figure 4.
23.
 A bowl of oatmeal topped with blueberries and 

In [167]:
with fitz.open(file_name) as doc:
    print(doc.load_page(19).)

HOW NUTRITION AFFECTS HEALTH 
The World Health Organization (WHO) defines health as “a state of complete physical, mental and 
social well-being and not merely the absence of disease or infirmity.”3 The WHO recognizes nutrition 
as a critical part of health and development, noting that better nutrition is related to:4 
• improved infant, child and maternal health 
• stronger immune systems 
• safer pregnancy and childbirth 
• lower risk of non-communicable diseases (such as type 2 diabetes and cardiovascular 
disease) 
• greater longevity 
• greater productivity, creating opportunities to break cycles of poverty and hunger 
Malnutrition, including both undernutrition and overnutrition, is a significant threat to human 
health. In fact, nutrition is associated with four of the top ten leading causes of death in the United 
States, including heart disease, cancer, diabetes, and stroke.5 
Figure 1.1. Age-adjusted death rates for the 10 leading causes of deaths: United States, 2016 and 201

In [ ]:
from langchain_community.document_loaders import UnstructuredPDFLoader

loader = UnstructuredPDFLoader(file_name)
docs = loader.load()

In [178]:
len(docs), len(docs[0].page_content)

(1, 1070583)

In [180]:
print(".\n".join(docs[0].page_content[:10000].split(".")))


Nutrition: Science and Everyday Application

Nutrition: Science and Everyday Application

V.
 2.
0

ALICE CALLAHAN, PHD; HEATHER LEONARD, MED, RDN; AND TAMBERLY POWELL, MS, RDN

Nutrition: Science and Everyday Application Copyright © 2020 by Alice Callahan, PhD; Heather Leonard, MEd, RDN; and Tamberly Powell, MS, RDN is licensed under a Creative Commons Attribution-NonCommercial 4.
0 International License, except where otherwise noted.


Contents

Introduction

About the Authors

Acknowledgements

Updates Made to OER

UNIT 1 – DESIGNING A HEALTHY DIET

Introduction to Designing a Healthy Diet

Nutrition and Health

An Introduction to Molecules

Classification of Nutrients

Defining Nutrient Requirements: Dietary Reference Intakes

Understanding Food Labels

Tools for Achieving a Healthy Diet

UNIT 2 — NUTRITION SCIENCE AND INFORMATION LITERACY

Introduction to Nutrition Science and Information Literacy

The Scientific Method

Types of Research Studies and How To Interpret Them

Healthy

In [193]:
text_splitter.split_text(docs[0].page_content)[0]


'Nutrition: Science and Everyday Application\n\nNutrition: Science and Everyday Application\n\nV. 2.0\n\nALICE CALLAHAN, PHD; HEATHER LEONARD, MED, RDN; AND TAMBERLY POWELL, MS, RDN\n\nNutrition: Science and Everyday Application Copyright © 2020 by Alice Callahan, PhD; Heather Leonard, MEd, RDN; and Tamberly Powell, MS, RDN is licensed under a Creative Commons Attribution-NonCommercial 4.0 International License, except where otherwise noted.\n\nContents\n\nIntroduction\n\nAbout the Authors\n\nAcknowledgements\n\nUpdates Made to OER\n\nUNIT 1 – DESIGNING A HEALTHY DIET\n\nIntroduction to Designing a Healthy Diet\n\nNutrition and Health\n\nAn Introduction to Molecules\n\nClassification of Nutrients\n\nDefining Nutrient Requirements: Dietary Reference Intakes\n\nUnderstanding Food Labels\n\nTools for Achieving a Healthy Diet\n\nUNIT 2 — NUTRITION SCIENCE AND INFORMATION LITERACY\n\nIntroduction to Nutrition Science and Information Literacy\n\nThe Scientific Method\n\nTypes of Research Stu